In [1]:
import pickle
models = []
models.append(pickle.load(open('finalized_model_nb.sav','rb')))
models.append(pickle.load(open('finalized_model_knn.sav','rb')))
models.append(pickle.load(open('finalized_model_DT.sav','rb')))
models.append(pickle.load(open('finalized_model_RF.sav','rb')))
models.append(pickle.load(open('finalized_model_xgb.sav','rb')))

In [2]:
import numpy as np
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [3]:
feat = np.array([[0,1.25,0,322]])

In [4]:
preds = []
for model in models:
    preds.append(model.predict(feat))

In [5]:
preds

[array([1], dtype=int64),
 array([1], dtype=int64),
 array([0], dtype=int64),
 array([0], dtype=int64),
 array([0], dtype=int64)]

In [6]:
for model in models:
    print(model)

GaussianNB()
KNeighborsClassifier(n_neighbors=3)
RandomizedSearchCV(cv=5, estimator=DecisionTreeClassifier(), n_iter=100,
                   n_jobs=1,
                   param_distributions={'max_depth': [3, 4, 5, 6, 8, 10, 12,
                                                      15],
                                        'max_features': ['auto', 'log2', 'sqrt',
                                                         None],
                                        'max_leaf_nodes': [None, 10, 20, 30, 40,
                                                           50, 60, 70],
                                        'min_samples_leaf': [1, 2, 3, 4, 5],
                                        'min_weight_fraction_leaf': [0.1, 0.2,
                                                                     0.3, 0.4],
                                        'splitter': ['best', 'random']},
                   random_state=10, verbose=2)
RandomizedSearchCV(cv=3, estimator=RandomForestClassifier()

In [7]:
def predict(feat,scaled_feat):
    preds = []
    i=0
    for model in models:
        if i<2:
            preds.append(model.predict(scaled_feat))
        else:
            preds.append(model.predict(feat))
        i+=1
    o = preds.count(1)
    z = preds.count(0)
    if(o>z):
        return 1
    else:
        return 0

In [8]:
import pandas as pd
df = pd.read_csv("feature_selection.csv")

In [9]:
X = df.drop(['Approved'],axis=1)
scaled_features = scaler.fit_transform(df.drop('Approved',axis=1))
y = df['Approved']

In [10]:
df_feat = pd.DataFrame(scaled_features,columns=df.columns[:-1])
df_feat.head()

,PriorDefault_t,YearsEmployed,CreditScore,Income
0,0.95465,-0.300502,-0.324711,-0.288910
1,0.95465,0.379591,1.079116,0.211684
2,0.95465,-0.205517,-0.605477,0.766248
3,0.95465,0.649348,0.798351,-0.958363
4,0.95465,-0.125730,-0.605477,-0.288910


In [11]:
pred = []
print(X.shape[0])
for i in range(X.shape[0]):
    feat = np.array([X.iloc[i].values])
    scaled_feat = np.array([df_feat.iloc[i].values])
    pred.append(predict(feat,scaled_feat))

690


In [12]:
from sklearn.metrics import confusion_matrix,classification_report,accuracy_score,roc_curve, roc_auc_score

In [13]:
accuracy_score(y,np.array(pred))

0.8724637681159421

In [14]:
confusion_matrix(y,np.array(pred))

array([[331,  52],
       [ 36, 271]], dtype=int64)

In [15]:
print(classification_report(y,np.array(pred)))

              precision    recall  f1-score   support

           0       0.90      0.86      0.88       383
           1       0.84      0.88      0.86       307

    accuracy                           0.87       690
   macro avg       0.87      0.87      0.87       690
weighted avg       0.87      0.87      0.87       690



In [16]:
model = pickle.load(open('finalized_model_knn.sav','rb'))

In [17]:
pred=[]
for i in range(X.shape[0]):
    scaled_feat = np.array([df_feat.iloc[i].values])
    pred.append(model.predict(scaled_feat))

In [18]:
accuracy_score(y,np.array(pred))

0.8826086956521739

In [19]:
confusion_matrix(y,np.array(pred))

array([[351,  32],
       [ 49, 258]], dtype=int64)

In [20]:
print(classification_report(y,np.array(pred)))

              precision    recall  f1-score   support

           0       0.88      0.92      0.90       383
           1       0.89      0.84      0.86       307

    accuracy                           0.88       690
   macro avg       0.88      0.88      0.88       690
weighted avg       0.88      0.88      0.88       690



In [21]:
TN, FP, FN, TP = confusion_matrix(y, pred).ravel()

In [22]:
specificity = TN / (TN+FP)
specificity

0.9164490861618799

In [23]:
recall = TP / (TP+FN)
recall

0.8403908794788274

In [24]:
precision = TP/(TP+FP)
precision

0.8896551724137931

In [25]:
f1score = (2*precision*recall)/(precision+recall)
f1score

0.864321608040201

# Therefore KNN with 3 neighbours has given best accuracy than bag of models.